# Black body spectra for stars

In [1]:
import astropy.units as u
from astropy.constants import c, h, k_B, b_wien, sigma_sb
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import math
import numpy as np
import astropy.units as u
%matplotlib inline

class BlackBody:
    def __init__(self,T):
        # A black body is fully defined by its temperature
        try:
            self.T = T.to(u.K)
        except AttributeError:
            self.T = T * u.K
        # We can use Wien's law to find the wavelength of the maximum
        self.lambda_max = (b_wien / self.T).to(u.nm)
        self.freq_max   = (c / self.lambda_max).to(u.Hz)
        # We can use Stefan Boltmann's law to get the total radiance
        self.total_radiance = sigma_sb * self.T**4
            
    def spec_radiance_freq(self, nu, units='cgs'):
        try:
            nu = nu.to(u.Hz)
        except AttributeError:
            nu *= u.Hz
        
        exponent  = (h * nu / k_B / self.T).decompose()
        prefactor = (2 * h * nu**3 / c**2).decompose()
        radiance  = prefactor / (np.exp(exponent) - 1)
        
        if   units=='cgs':
            radiance = radiance.to(u.erg / u.s / u.cm**2 / u.Hz)
        elif units=='SI':
            radiance = radiance.to(u.W / u.m**2 / u.Hz)
        
        return radiance
    
    def spec_radiance_wavelenght(self, lamb, units='cgs'):
        try:
            lamb = lamb.to(u.nm)
        except AttributeError:
            lamb *= u.nm
            
        exponent  = (h * c / lamb / k_B / self.T).decompose()
        prefactor = (2 * h * c**2 / lamb**5).decompose()
        radiance  = prefactor / (np.exp(exponent) - 1)
        
        if units=='cgs':
            radiance = radiance.to(u.erg / u.s / u.cm**2 / u.nm)
        elif units=='SI':
            radiance = radiance.to(u.W / u.m**2 / u.nm)
            
        return radiance

def star_color(T):
    
    # Temperature information by spectral type: https://sites.uni.edu/morgans/astro/course/Notes/section2/spectraltemps.html
    # Color information by spectral type: http://www.vendian.org/mncharity/dir3/starcolor/
    Type = ['O5',  'B1',  'B3',  'B5',  'B8',  'A1', 'A3', 'A5', 'F0', 'F2', 'F5', 'F8', 'G2', 'G5', 'G8', 'K0', 'K4', 'K7', 'M2', 'M4', 'M6']
    T_s  = [54000, 23000, 21000, 15200, 12300, 9330, 8750, 8310, 7350, 7050, 6700, 6300, 5800, 5660, 5440, 5240, 4600, 4000, 3600, 3400, 3100]
    R_s  = [157,   162,   167,   170,   175,   186,  192,  202,  228,  237,  251,  255,  255,  255,  255,  255,  255,  255,  255,  255,  255] 
    G_s  = [180,   185,   188,   191,   195,   204,  209,  216,  232,  238,  248,  249,  245,  244,  241,  235,  215,  198,  190,  187,  187]
    B_s  = [255,   255,   255,   255,   255,   255,  255,  255,  255,  255,  255,  249,  236,  232,  223,  209,  174,  144,  127,  123,  123]
    
    # Need to order Ts for interpolation to work
    T_s, R_s, G_s, B_s = T_s[::-1], R_s[::-1], G_s[::-1], B_s[::-1]
    R = np.interp(T, T_s, R_s)
    G = np.interp(T, T_s, G_s)
    B = np.interp(T, T_s, B_s)
    
    # Normalize
    R /= 255
    G /= 255
    B /= 255
    
    return (R, G, B)

def star_comp(T1, T2, R1, R2, solar_units=False, log_scale=False, lambda_min=1e-1, y_min=-1):
    
    # hard-coded stuff
    T_sun = 5778
    R_sun = 6.957e10 * u.cm
    L_sun = 3.846e33 * u.erg / u.s
    
    # define the blackbodies
    bb_1 = BlackBody(T1)
    bb_2 = BlackBody(T2)
    
    a_1 = 4 * math.pi * (R1*R_sun)**2
    a_2 = 4 * math.pi * (R2*R_sun)**2
    
    # get the spectra
    lambdas = (np.linspace(lambda_min, 3, 200) * u.micron).to(u.nm)
    rad_1   = bb_1.spec_radiance_wavelenght(lambdas, units='cgs')
    rad_2   = bb_2.spec_radiance_wavelenght(lambdas, units='cgs')
    
    spec_1 = rad_1 * a_1
    spec_2 = rad_2 * a_2
    
    if solar_units:
        spec_1 /= L_sun
        spec_2 /= L_sun
    
    # Set up the plot
    fontsize = 20
    matplotlib.rcParams.update({'font.size': fontsize})
    fig, axs = plt.subplots(ncols=2, nrows=1, gridspec_kw=dict(width_ratios=[2,1]), figsize=(20,10))
    
    
    # Plot spectrum
    axs[0].plot(lambdas, spec_1, linewidth=3, color=star_color(T1),        alpha=1.00, label=r"Star 1")
    axs[0].axvline(bb_1.lambda_max.value, linewidth=3, linestyle='--', color=star_color(T1))
    axs[0].plot(lambdas, spec_2, linewidth=3, color=star_color(T2), alpha=1.00, label=r"Star 2")
    axs[0].axvline(bb_2.lambda_max.value, linewidth=3, linestyle='--', color=star_color(T2))
    
    axs[0].set_facecolor('black')
    axs[0].legend(loc=1, fontsize=fontsize)
    axs[0].set_xlabel(r"Wavelength [nm]", fontsize=fontsize)
    if solar_units:
        axs[0].set_ylabel(r"Spectral radiance [$L_{\odot}$/ nm]", fontsize=fontsize)
    else:
        axs[0].set_ylabel(r"Spectral radiance [erg / s / nm]", fontsize=fontsize)
    if log_scale:
        axs[0].set_yscale('log')
        if y_min != -1:
            axs[0].set_ylim(bottom=y_min)
    
    # Plot stars
    gap = (R1 + R2) / 10
    R_max = max(R1, R2)
    axs[1].set_aspect('equal')
    axs[1].add_artist(Circle((0,0),          radius=R1, facecolor=star_color(T1)))
    axs[1].add_artist(Circle((0, R1+R2+gap), radius=R2, facecolor=star_color(T2)))
    axs[1].set_xlim(-R_max-gap, R_max+gap)
    axs[1].set_ylim(-R1-gap, R1+2*R2+2*gap)
    axs[1].set_facecolor('black')
    axs[1].get_xaxis().set_visible(False)
    axs[1].get_yaxis().set_visible(False)
    
    return


In [2]:
# Code to do a demo in class on the effec of the temperature of a black body on
# its spectral radiance

from ipywidgets import interactive

star_comp_demo = interactive(star_comp, T1=(3000, 20000, 100), T2=(3000, 20000, 100), 
                             R1=(0.001, 70, 0.1), R2=(0.001, 70, 0.1), solar_units=False, log_scale=False)
output  = star_comp_demo.children[-1]
star_comp_demo

interactive(children=(IntSlider(value=11500, description='T1', max=20000, min=3000, step=100), IntSlider(value…